In [ ]:
# Make sure that Python starts in Workflow-folder or else the modules will be screewed up!
import sys
import os
import getpass
import datetime

# Something about adding a thing to Python path.
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# What's the time?
today = datetime.date.today()

# Identify user
user = getpass.getuser()
if user == "sidsel":
    parquet_path = "/home/sidsel/workspace/sparkdata/parquet"
elif user == "svanhmic":
    parquet_path = "/home/svanhmic/workspace/data/DABAI/sparkdata/parquet"
    
# Start the logger.
import logging
logger_tester = logging.getLogger(__name__)
logger_tester.setLevel(logging.INFO)
logger_file_handler_param = logging.FileHandler('/tmp/workflow_test_cleaning_{!s}.log'.format(today.strftime('%d-%m-%Y')))
logger_formatter_param = logging.Formatter('%(asctime)s;%(levelname)s;%(name)s;%(message)s')

logger_tester.addHandler(logger_file_handler_param)
logger_file_handler_param.setFormatter(logger_formatter_param)

In [ ]:
from shared.Extension_to_timeit import pretty_time_result
from shared.GeneralDataImport import GeneralDataImport

dataIO = GeneralDataImport(parquet_path+"/normal_cluster_n_1000.parquet")

In [ ]:
dataIO.select_columns()

In [ ]:
df = dataIO.data_frame
df.show(3)

In [ ]:
#ax = sb.regplot('a','b',df.toPandas(),fit_reg=False)
import seaborn as sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
ax = sb.lmplot('a',
               'b',
               dataIO.data_frame.toPandas(),
               fit_reg=False,
               size=8,
               hue='k',
               scatter_kws={'alpha':0.7,'s':60}
              )
plt.show()

In [ ]:
#import data!
#from pyspark.sql import functions as F
#from shared.create_dummy_data import create_dummy_data 

#test_timer = %timeit -o feature_data = create_dummy_data(1000, "x y z", "label", outlier_number=0.2, outlier_factor=20)
#feature_data = feature_data.select([(10*F.col(i)).alias(i) for i in ["x","y","z"]])
#feature_data.orderBy('x',ascending=[0,0,0]).show()

In [ ]:
# Select parameters
from cleaning.CreateParametersCleaning import ParamsCleaning

params = ParamsCleaning()
parameters = params.select_parameters()

In [ ]:
parameters

In [ ]:
#print(params.output_parameters(parameters))
#test_params_1 = {'tol': 0.00001, 'k': 3, 'maxIter': 300, 'algorithm': 'GaussianMixture', 'seed': 1080866016001745000}
test_params_1 = params.output_parameters(parameters)
print(test_params_1)

In [ ]:
#print(sc.defaultMinPartitions)
#print(sc.defaultParallelism)
#conf = sc.getConf()
#conf.getAll()

In [ ]:
from cleaning.ExecuteCleaningWorkflow import ExecuteWorkflow

partitions = [80]
sizes = [1000]
features = list(map(lambda x: x.name,dataIO.list_features))
labels = list(map(lambda x: x.name,dataIO.list_label))

#print(features)
#print(labels)

execution_model = ExecuteWorkflow(
    dict_params = test_params_1,
    cols_features = features, 
    cols_labels = labels)

    
collection_of_data = [parquet_path+'/normal_cluster_n_{}.parquet'.format(i) for i in sizes]
collection_of_model = []
collection_of_transformed = []
#collection_of_data
#counts = [i.rdd.getNumPartitions() for i in collection_of_data]
#counts
#collection_of_data.append(df)

In [ ]:
for jdx, partition_size in enumerate(partitions):
    for idx, data in enumerate(collection_of_data):

        df_data = (spark.
                   read.
                   parquet(data).
                   repartition(partition_size)
                  )
        
        iteration = idx+jdx*len(collection_of_data)
        logger_tester.info(
            'Iteration {} for data size {}'.
            format(iteration, sizes[idx])
        )

        model_timer = %timeit -r1 -o collection_of_model.append(execution_model.execute_pipeline(df_data)) 
        transformer_timer = %timeit -o execution_model.apply_model(collection_of_model[iteration],df_data)
        collection_of_model = collection_of_model[:iteration+1]
        logger_tester.info(
            'Iteration '+str(iteration)+' for training model : '+pretty_time_result(model_timer))
        logger_tester.info(
            'Iteration '+str(iteration)+' for transforming model : '+pretty_time_result(transformer_timer))
        #merged_df.write.parquet('/home/svanhmic/workspace/data/DABAI/sparkdata/parquet/merged_df_parquet')

In [ ]:
df_data = (spark.
           read.
           parquet(data).
           repartition(partition_size)
          )

df_results = execution_model.apply_model(collection_of_model[iteration],df_data)
df.show(3)

In [ ]:
from cleaning.ShowCleaning import ShowResults
results = ShowResults(
    execution_model.parameters,
    list_features=execution_model.features,
    list_labels=execution_model.labels)

In [ ]:
df_with_dists = results.select_prototypes(df_results)

In [ ]:
from shared import Plot2DGraphs
Plot2DGraphs.plot_gaussians(
    df_results,
    execution_model.features,
    gaussian_std=1)

In [ ]:
ax = sb.lmplot('a',
               'b',
               dataIO.data_frame.toPandas(),
               fit_reg=False,
               size=8,
               hue='k',
               scatter_kws={'alpha':0.7,'s':60}
              )
plt.show()